# Traditional ML Baseline for ABSA Sentiment Classification

**Objective:** Train Random Forest classifier with TF-IDF features as baseline to compare with XLM-RoBERTa

**Academic Justification:**
- Establishes baseline performance using classical ML (Scikit-learn Random Forest)
- TF-IDF captures term importance without contextual embeddings
- Comparison validates whether transformer pre-training provides value for Manglish code-switching
- Following best practices: same train/test split, same evaluation protocol, same class imbalance handling

# Stage 0: Environment Setup

In [ ]:
# Connect to google drive
from google.colab import drive
import os

# 1. Mount Google Drive (To save the model checkpoints)
drive.mount('/content/drive')

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

# NLP Processing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# ML & Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("✓ All libraries loaded successfully")

✓ All libraries loaded successfully


In [2]:
# Download NLTK resources (run once)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('punkt_tab')

print("✓ NLTK resources downloaded")

✓ NLTK resources downloaded


[nltk_data] Downloading package punkt_tab to C:\Users\Ong Hui
[nltk_data]     Ling\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Stage 1: Configuration & Data Loading

In [3]:
# Configuration
#DATA_PATH = r'/content/drive/MyDrive/Aspect-Based-Sentiment-Analysis/Dataset/aspect_categorization_after_filtering.pkl'  
#GOLD_PATH = r'/content/drive/MyDrive/Aspect-Based-Sentiment-Analysis/Dataset/Final_Gold_Standard.csv'
DATA_PATH = r'C:\Users\Ong Hui Ling\Dropbox\PC\Documents\Github\Aspect-Based-Sentiment-Analysis\Dataset\aspect_categorization_after_filtering.pkl' 
GOLD_PATH = r'C:\Users\Ong Hui Ling\Dropbox\PC\Documents\Github\Aspect-Based-Sentiment-Analysis\Dataset\Final_Gold_Standard.csv'
RANDOM_SEED = 42
TEST_SIZE = 0.15
VAL_SIZE = 0.10

# Label encoding
LABEL2ID = {"negative": 0, "positive": 1}
ID2LABEL = {v: k for k, v in LABEL2ID.items()}

print(f"✓ Configuration loaded")
print(f"  Data path: {DATA_PATH}")
print(f"  Gold path: {GOLD_PATH}")
print(f"  Random seed: {RANDOM_SEED}")

✓ Configuration loaded
  Data path: C:\Users\Ong Hui Ling\Dropbox\PC\Documents\Github\Aspect-Based-Sentiment-Analysis\Dataset\aspect_categorization_after_filtering.pkl
  Gold path: C:\Users\Ong Hui Ling\Dropbox\PC\Documents\Github\Aspect-Based-Sentiment-Analysis\Dataset\Final_Gold_Standard.csv
  Random seed: 42


In [4]:
print("="*70)
print("LOADING & PREPARING DATA")
print("="*70)

# Load training data
df = pd.read_pickle(DATA_PATH)
print(f"  Raw segments loaded: {len(df):,}")

# --- PREVENT DATA LEAKAGE: Exclude gold standard review IDs ----------
print(f"\n  ⚠️  DATA LEAKAGE PREVENTION:")
print(f"  Loading gold standard to identify held-out review IDs...")

try:
    gold_df = pd.read_csv(GOLD_PATH)
    
    # Extract unique Original_Review_IDs from gold dataset
    if 'Original_Review_ID' in gold_df.columns:
        gold_review_ids = set(gold_df['Original_Review_ID'].unique())
    elif 'Review_ID' in gold_df.columns:
        gold_review_ids = set(gold_df['Review_ID'].unique())
    else:
        print(f"  ⚠️  Warning: Could not find review ID column in gold dataset")
        gold_review_ids = set()
    
    print(f"  ✓ Gold dataset loaded: {len(gold_df):,} annotations")
    print(f"  ✓ Unique review IDs in gold: {len(gold_review_ids):,}")
    
    # Filter out segments from gold review IDs
    n_before = len(df)
    df = df[~df['Original_Review_ID'].isin(gold_review_ids)].copy()
    n_after = len(df)
    n_removed = n_before - n_after
    
    print(f"  ✓ Filtered out {n_removed:,} segments from gold reviews ({n_removed/n_before*100:.1f}%)")
    print(f"  ✓ Training segments remaining: {n_after:,}")
    
except Exception as e:
    print(f"  ✗ Error loading gold dataset: {e}")

# Filter to single-aspect segments
df["num_aspects"] = df["Aspect_Labels"].apply(len)
df_single = df[df["num_aspects"] == 1].copy()
df_single["aspect"] = df_single["Aspect_Labels"].apply(lambda x: x[0])

n_multi = len(df) - len(df_single)
pct_retained = (len(df_single) / len(df)) * 100

print(f"\n  FILTERING STRATEGY:")
print(f"    Single-aspect segments:  {len(df_single):>7,} ({pct_retained:>5.1f}%) → KEPT")
print(f"    Multi-aspect segments:   {n_multi:>7,} ({100-pct_retained:>5.1f}%) → DROPPED")

# Encode labels
df_single["label"] = df_single["Sentiment_Label"].map(LABEL2ID)

print(f"\n  Label distribution:")
for label_name, label_id in LABEL2ID.items():
    count = (df_single["label"] == label_id).sum()
    pct = count / len(df_single) * 100
    print(f"    {label_name:<10}: {count:>7,} ({pct:>5.1f}%)")

print(f"\n✓ Data preparation complete: {len(df_single):,} segments ready for training")

LOADING & PREPARING DATA
  Raw segments loaded: 128,778

  ⚠️  DATA LEAKAGE PREVENTION:
  Loading gold standard to identify held-out review IDs...
  ✓ Gold dataset loaded: 645 annotations
  ✓ Unique review IDs in gold: 131
  ✓ Filtered out 996 segments from gold reviews (0.8%)
  ✓ Training segments remaining: 127,782

  FILTERING STRATEGY:
    Single-aspect segments:   97,911 ( 76.6%) → KEPT
    Multi-aspect segments:    29,871 ( 23.4%) → DROPPED

  Label distribution:
    negative  :   6,498 (  6.6%)
    positive  :  91,413 ( 93.4%)

✓ Data preparation complete: 97,911 segments ready for training


# Stage 2: Text Preprocessing (Conventional NLP)

In [5]:
def preprocess_text(text):
    """
    Apply conventional NLP preprocessing pipeline.
    
    Steps:
    1. Lowercase conversion
    2. Remove special characters (keep letters, numbers, spaces)
    3. Tokenization
    4. Remove English stopwords
    5. Lemmatization
    
    Why:
        Traditional ML models (Random Forest, SVM) lack contextual understanding.
        Preprocessing reduces noise and dimensionality for TF-IDF vectorization.
    
    Note:
        We do NOT remove Manglish terms (sedap, mamak) as they carry sentiment.
        Stopwords removal is conservative to preserve sentiment-bearing phrases.
    """
    if pd.isna(text):
        return ""
    
    # Lowercase
    text = text.lower()
    
    # Remove special characters but keep spaces
    text = re.sub(r'[^a-z0-9\s]', '', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join back
    return ' '.join(tokens)

# Test preprocessing
sample_text = "The nasi lemak was incredibly sedap but the service was lambat!"
print(f"Original:  {sample_text}")
print(f"Processed: {preprocess_text(sample_text)}")

Original:  The nasi lemak was incredibly sedap but the service was lambat!
Processed: nasi lemak incredibly sedap service lambat


In [6]:
print("="*70)
print("PREPROCESSING TEXT DATA")
print("="*70)

# Apply preprocessing to all segments
print(f"Processing {len(df_single):,} segments...")
df_single['processed_text'] = df_single['Segment'].apply(preprocess_text)




PREPROCESSING TEXT DATA
Processing 97,911 segments...


In [7]:
# Check for empty processed texts
print("="*70)
print("EMPTY PROCESSED TEXTS ORIGNIAL SEGMENTS")
print("="*70)

df_single[df_single['processed_text'].str.len() == 0]['Segment'].value_counts()

EMPTY PROCESSED TEXTS ORIGNIAL SEGMENTS


Segment
again                 46
once again             4
over again             2
again)                 1
again not 豬油渣          1
again ❤️😊              1
here again             1
was again              1
because again          1
will be here again     1
i was here again       1
so soâ€¦               1
Name: count, dtype: int64

In [8]:
# Check for empty processed texts
n_empty = (df_single['processed_text'].str.len() == 0).sum()

if n_empty > 0:
    print(f"  ⚠️  Warning: {n_empty} segments became empty after preprocessing")
    df_single = df_single[df_single['processed_text'].str.len() > 0].copy()
    print(f"  ✓ Removed empty segments. Remaining: {len(df_single):,}")

print(f"\n✓ Text preprocessing complete")
print(f"\nSample processed segments:")
for i in range(3):
    print(f"\n  Original:  {df_single.iloc[i]['Segment'][:80]}...")
    print(f"  Processed: {df_single.iloc[i]['processed_text'][:80]}...")

  ⚠️  Warning: 61 segments became empty after preprocessing
  ✓ Removed empty segments. Remaining: 97,850

✓ Text preprocessing complete

Sample processed segments:

  Original:  coconut cream - the perfect finish to the meal...
  Processed: coconut cream perfect finish meal...

  Original:  cooked perfectly...
  Processed: cooked perfectly...

  Original:  don’t miss the sweet appam with brown sugar...
  Processed: dont miss sweet appam brown sugar...


# Stage 3: Train/Val/Test Split 

In [9]:
print("="*70)
print("TRAIN/VAL/TEST SPLIT")
print("="*70)

# Stage 1: Separate test set (stratified)
df_trainval, df_test = train_test_split(
    df_single,
    test_size=TEST_SIZE,
    stratify=df_single["label"],
    random_state=RANDOM_SEED
)

# Stage 2: Split remainder into train + val
adjusted_val_size = VAL_SIZE / (1.0 - TEST_SIZE)
df_train, df_val = train_test_split(
    df_trainval,
    test_size=adjusted_val_size,
    stratify=df_trainval["label"],
    random_state=RANDOM_SEED
)

print(f"\nSplit sizes:")
for name, split_df in [("Train", df_train), ("Val", df_val), ("Test", df_test)]:
    pos = (split_df["label"] == 1).sum()
    neg = (split_df["label"] == 0).sum()
    print(f"  {name:<6}: {len(split_df):>7,} rows | "
          f"pos: {pos:>6,} ({pos/len(split_df)*100:>5.1f}%) | "
          f"neg: {neg:>6,} ({neg/len(split_df)*100:>5.1f}%)")

# Extract X (processed text) and y (labels)
X_train = df_train['processed_text'].values
X_val = df_val['processed_text'].values
X_test = df_test['processed_text'].values

y_train = df_train['label'].values
y_val = df_val['label'].values
y_test = df_test['label'].values

print(f"\n✓ Data split complete")

TRAIN/VAL/TEST SPLIT

Split sizes:
  Train :  73,387 rows | pos: 68,519 ( 93.4%) | neg:  4,868 (  6.6%)
  Val   :   9,785 rows | pos:  9,136 ( 93.4%) | neg:    649 (  6.6%)
  Test  :  14,678 rows | pos: 13,704 ( 93.4%) | neg:    974 (  6.6%)

✓ Data split complete


# Stage 4: TF-IDF Vectorization

In [10]:
print("="*70)
print("TF-IDF VECTORIZATION")
print("="*70)

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer(
    max_features=5000,     # Limit to top 5000 features (prevent overfitting)
    ngram_range=(1, 2),    # Unigrams + bigrams (capture phrases like "nasi lemak")
    min_df=2,              # Ignore terms appearing in < 2 documents
    max_df=0.8,            # Ignore terms appearing in > 80% of documents
    sublinear_tf=True      # Apply sublinear tf scaling (1 + log(tf))
)

print(f"TF-IDF Configuration:")
print(f"  Max features:  {vectorizer.max_features:,}")
print(f"  N-gram range:  {vectorizer.ngram_range}")
print(f"  Min doc freq:  {vectorizer.min_df}")
print(f"  Max doc freq:  {vectorizer.max_df}")

# Fit on training data and transform all splits
print(f"\nFitting TF-IDF on {len(X_train):,} training samples...")
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

print(f"\n✓ TF-IDF vectorization complete")
print(f"  Train shape: {X_train_tfidf.shape}")
print(f"  Val shape:   {X_val_tfidf.shape}")
print(f"  Test shape:  {X_test_tfidf.shape}")
print(f"  Vocabulary size: {len(vectorizer.vocabulary_):,} terms")

# Show top features by TF-IDF score
feature_names = vectorizer.get_feature_names_out()
tfidf_scores = X_train_tfidf.sum(axis=0).A1
top_indices = tfidf_scores.argsort()[-20:][::-1]

print(f"\nTop 20 features by TF-IDF score:")
for idx in top_indices:
    print(f"  {feature_names[idx]:<20} {tfidf_scores[idx]:.2f}")

TF-IDF VECTORIZATION
TF-IDF Configuration:
  Max features:  5,000
  N-gram range:  (1, 2)
  Min doc freq:  2
  Max doc freq:  0.8

Fitting TF-IDF on 73,387 training samples...

✓ TF-IDF vectorization complete
  Train shape: (73387, 5000)
  Val shape:   (9785, 5000)
  Test shape:  (14678, 5000)
  Vocabulary size: 5,000 terms

Top 20 features by TF-IDF score:
  food                 2059.51
  service              1339.57
  good                 1215.44
  delicious            1155.81
  friendly             1032.15
  staff                913.08
  chicken              826.22
  taste                804.06
  price                803.12
  nice                 774.98
  fresh                739.35
  tasty                663.22
  great                639.46
  place                630.93
  attentive            630.19
  dish                 605.45
  restaurant           541.72
  come                 531.13
  clean                509.03
  rice                 504.46


# Stage 5: Multi-Model Training 

**Academic Justification:**
- Compare 5 traditional ML classifiers: Logistic Regression, SVM, Naive Bayes, XGBoost, Random Forest
- Evaluate on validation set to identify best model(s) for hyperparameter tuning
- Different models capture different patterns: linear (LR, SVM) vs non-linear (RF, XGB) vs probabilistic (NB)
- Best practice: broad comparison before expensive hyperparameter tuning

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
import time

print("="*70)
print("TRAINING MULTIPLE ML MODELS")
print("="*70)

# Compute class weights (for models that support it)
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = {i: class_weights_array[i] for i in range(len(class_weights_array))}

# Calculate scale_pos_weight for XGBoost (ratio of negative to positive)
scale_pos_weight = class_weights_array[0] / class_weights_array[1]

print(f"Class imbalance handling:")
print(f"  Negative weight: {class_weights_dict[0]:.4f}")
print(f"  Positive weight: {class_weights_dict[1]:.4f}")
print(f"  XGBoost scale_pos_weight: {scale_pos_weight:.4f}")

# Define 5 models with reasonable default parameters
models = {
    "Logistic Regression": LogisticRegression(
        max_iter=1000,
        class_weight=class_weights_dict,
        random_state=RANDOM_SEED,
        n_jobs=-1
    ),
    "SVM (Linear)": SVC(
        kernel='linear',
        class_weight=class_weights_dict,
        random_state=RANDOM_SEED
    ),
    "Naive Bayes": MultinomialNB(
        alpha=1.0  
    ),
    "XGBoost": XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        scale_pos_weight=scale_pos_weight,
        random_state=RANDOM_SEED,
        n_jobs=-1,
        eval_metric='logloss'
    ),
    "Random Forest": RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        class_weight=class_weights_dict,
        random_state=RANDOM_SEED,
        n_jobs=-1
    )
}

print(f"\n{'='*70}")
print(f"TRAINING {len(models)} MODELS")
print(f"{'='*70}\n")

# Train all models and store results
trained_models = {}
training_times = {}

for model_name, model in models.items():
    print(f"Training {model_name}...")
    start_time = time.time()
    
    model.fit(X_train_tfidf, y_train)
    
    training_time = time.time() - start_time
    trained_models[model_name] = model
    training_times[model_name] = training_time
    
    print(f"  ✓ Complete in {training_time:.2f} seconds\n")

print(f"✓ All models trained successfully")

TRAINING MULTIPLE ML MODELS
Class imbalance handling:
  Negative weight: 7.5377
  Positive weight: 0.5355
  XGBoost scale_pos_weight: 14.0754

TRAINING 5 MODELS

Training Logistic Regression...
  ✓ Complete in 2.78 seconds

Training SVM (Linear)...
  ✓ Complete in 307.85 seconds

Training Naive Bayes...
  ✓ Complete in 0.01 seconds

Training XGBoost...
  ✓ Complete in 6.69 seconds

Training Random Forest...
  ✓ Complete in 3.35 seconds

✓ All models trained successfully


# Stage 6: Model Comparison on Validation Set

In [12]:
print("="*70)
print("COMPARING MODELS ON VALIDATION SET")
print("="*70)

# Evaluate all models on validation set
results = []

def compute_metrics(y_true, y_pred):
    """Compute same metrics as BERT for fair comparison."""
    acc = accuracy_score(y_true, y_pred)
    macro_f1 = f1_score(y_true, y_pred, average="macro")
    per_class_f1 = f1_score(y_true, y_pred, average=None, labels=[0, 1])
    
    return {
        "accuracy": round(acc, 4),
        "macro_f1": round(macro_f1, 4),
        "neg_f1": round(per_class_f1[0], 4),
        "pos_f1": round(per_class_f1[1], 4),
    }

for model_name, model in trained_models.items():
    # Predict on validation set
    y_pred_val = model.predict(X_val_tfidf)
    
    # Compute metrics
    val_metrics = compute_metrics(y_val, y_pred_val)
    
    results.append({
        'Model': model_name,
        'Accuracy': val_metrics['accuracy'],
        'Macro-F1': val_metrics['macro_f1'],
        'Negative F1': val_metrics['neg_f1'],
        'Positive F1': val_metrics['pos_f1'],
        'Training Time (s)': training_times[model_name]
    })

# Create results DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Macro-F1', ascending=False)

print(f"\n{'='*70}")
print(f"VALIDATION SET RESULTS (Ranked by Macro-F1)")
print(f"{'='*70}\n")
print(results_df.to_string(index=False))

COMPARING MODELS ON VALIDATION SET

VALIDATION SET RESULTS (Ranked by Macro-F1)

              Model  Accuracy  Macro-F1  Negative F1  Positive F1  Training Time (s)
Logistic Regression    0.8239    0.6236       0.3491       0.8982           2.781038
       SVM (Linear)    0.8056    0.6062       0.3260       0.8864         307.845950
        Naive Bayes    0.9388    0.5905       0.2129       0.9682           0.011084
            XGBoost    0.9359    0.5203       0.0739       0.9668           6.688418
      Random Forest    0.6003    0.4787       0.2269       0.7305           3.353922


# Stage 7: Model Evaluation on Test Set

In [13]:
print("="*70)
print("COMPARING MODELS ON TEST SET")
print("="*70)

# Evaluate all models on test set
results = []

for model_name, model in trained_models.items():
    # Predict on test set
    y_pred_test = model.predict(X_test_tfidf)
    
    # Compute metrics
    test_metrics = compute_metrics(y_test, y_pred_test)
    
    results.append({
        'Model': model_name,
        'Accuracy': test_metrics['accuracy'],
        'Macro-F1': test_metrics['macro_f1'],
        'Negative F1': test_metrics['neg_f1'],
        'Positive F1': test_metrics['pos_f1'],
        'Training Time (s)': training_times[model_name]
    })

# Create results DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Macro-F1', ascending=False)

print(f"\n{'='*70}")
print(f"TEST SET RESULTS (Ranked by Macro-F1)")
print(f"{'='*70}\n")
print(results_df.to_string(index=False))

COMPARING MODELS ON TEST SET

TEST SET RESULTS (Ranked by Macro-F1)

              Model  Accuracy  Macro-F1  Negative F1  Positive F1  Training Time (s)
Logistic Regression    0.8193    0.6233       0.3516       0.8950           2.781038
        Naive Bayes    0.9419    0.6137       0.2576       0.9698           0.011084
       SVM (Linear)    0.8023    0.6097       0.3356       0.8839         307.845950
            XGBoost    0.9368    0.5296       0.0920       0.9672           6.688418
      Random Forest    0.6032    0.4809       0.2288       0.7329           3.353922


# Stage 8: Gold Standard Evaluation (Before Tuning)

In [14]:
print("="*70)
print("EVALUATING ON GOLD STANDARD (GROUND TRUTH)")
print("="*70)

# Load gold standard
gold_df = pd.read_csv(GOLD_PATH)
print(f"  Gold dataset loaded: {len(gold_df):,} rows")

# Prepare gold data (same as BERT)
gold_df_prep = gold_df.copy()
gold_df_prep.rename(columns={
    "Manual_Aspect": "aspect",
    "Manual_Sentiment": "Sentiment_Label",
}, inplace=True)

# Normalize sentiment labels to lowercase
gold_df_prep["Sentiment_Label"] = gold_df_prep["Sentiment_Label"].str.lower()

# Handle multi-aspect segments: explode into separate rows
import ast
def parse_aspect(val):
    if isinstance(val, str):
        try:
            parsed = ast.literal_eval(val)
            if isinstance(parsed, list):
                return parsed
            else:
                return [parsed]
        except (ValueError, SyntaxError):
            return [val]
    elif isinstance(val, list):
        return val
    else:
        return [str(val)]

gold_df_prep["aspect"] = gold_df_prep["aspect"].apply(parse_aspect)
gold_df_exploded = gold_df_prep.explode("aspect").reset_index(drop=True)

print(f"  After exploding: {len(gold_df_exploded):,} aspect-segment pairs")

# Encode labels
gold_df_exploded["label"] = gold_df_exploded["Sentiment_Label"].map(LABEL2ID)

# Preprocess gold text
print(f"\nPreprocessing gold segments...")
gold_df_exploded['processed_text'] = gold_df_exploded['Segment'].apply(preprocess_text)

# Vectorize gold text
print(f"Vectorizing gold segments...")
X_gold = gold_df_exploded['processed_text'].values
X_gold_tfidf = vectorizer.transform(X_gold)
y_gold = gold_df_exploded['label'].values

# Predict on gold using best model 
print(f"Running inference on gold set using ...")

# Evaluate all models on gold data set
results = []

for model_name, model in trained_models.items():
    # Predict on test set
    y_pred_gold = model.predict(X_gold_tfidf)
    
    # Compute metrics
    gold_metrics = compute_metrics(y_gold, y_pred_gold)
    
    results.append({
        'Model': model_name,
        'Accuracy': gold_metrics['accuracy'],
        'Macro-F1': gold_metrics['macro_f1'],
        'Negative F1': gold_metrics['neg_f1'],
        'Positive F1': gold_metrics['pos_f1'],
        'Training Time (s)': training_times[model_name]
    })

# Create results DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Macro-F1', ascending=False)

print(f"\n{'='*70}")
print(f"GOLD SET RESULTS (Ranked by Macro-F1)")
print(f"{'='*70}\n")
print(results_df.to_string(index=False))

EVALUATING ON GOLD STANDARD (GROUND TRUTH)
  Gold dataset loaded: 645 rows
  After exploding: 721 aspect-segment pairs

Preprocessing gold segments...
Vectorizing gold segments...
Running inference on gold set using ...

GOLD SET RESULTS (Ranked by Macro-F1)

              Model  Accuracy  Macro-F1  Negative F1  Positive F1  Training Time (s)
Logistic Regression    0.8350    0.8197       0.7671       0.8722           2.781038
       SVM (Linear)    0.8280    0.8135       0.7615       0.8655         307.845950
      Random Forest    0.7642    0.7591       0.7240       0.7942           3.353922
        Naive Bayes    0.6755    0.5258       0.2595       0.7922           0.011084
            XGBoost    0.6477    0.4601       0.1419       0.7784           6.688418


In [16]:
# Identify best model
best_model_name = results_df.iloc[0]['Model']
best_macro_f1 = results_df.iloc[0]['Macro-F1']

# Store best model for hyperparameter tuning
best_model = trained_models[best_model_name]

print(f"\n{'='*70}")
print(f"🏆 BEST MODEL: {best_model_name}")

# Best model prediction results
y_pred_gold = best_model.predict(X_gold_tfidf)
    
# Compute metrics
gold_metrics = compute_metrics(y_gold, y_pred_gold)

print(f"★ GOLD TEST SET RESULTS - {best_model_name} (Before Tuning)")
print(f"★ Total samples: {len(gold_df_exploded):,} aspect-segment pairs")
print(f"{'='*70}")
print(f"\nOVERALL PERFORMANCE:")
print(f"  Accuracy:  {gold_metrics['accuracy']:.4f}")
print(f"  Macro-F1:  {gold_metrics['macro_f1']:.4f}")
print(f"    Negative F1: {gold_metrics['neg_f1']:.4f}")
print(f"    Positive F1: {gold_metrics['pos_f1']:.4f}")

print(f"\n⚠️  Note: This is baseline performance before hyperparameter tuning")


🏆 BEST MODEL: Logistic Regression
★ GOLD TEST SET RESULTS - Logistic Regression (Before Tuning)
★ Total samples: 721 aspect-segment pairs

OVERALL PERFORMANCE:
  Accuracy:  0.8350
  Macro-F1:  0.8197
    Negative F1: 0.7671
    Positive F1: 0.8722

⚠️  Note: This is baseline performance before hyperparameter tuning


In [17]:
# Per-aspect breakdown
print(f"\nPER-ASPECT BREAKDOWN (for comparison with BERT):")
print(f"{'Aspect':<20} {'Samples':>8} {'Accuracy':>10} {'Macro-F1':>10}")
print(f"{'-'*20} {'-'*8} {'-'*10} {'-'*10}")

aspects_unique = sorted(gold_df_exploded["aspect"].unique())
for aspect in aspects_unique:
    mask = gold_df_exploded["aspect"] == aspect
    y_aspect = y_gold[mask]
    y_pred_aspect = y_pred_gold[mask]
    
    try:
        metrics = compute_metrics(y_aspect, y_pred_aspect)
        n_samples = mask.sum()
        print(f"{aspect:<20} {n_samples:>8} {metrics['accuracy']:>10.4f} {metrics['macro_f1']:>10.4f}")
    except:
        print(f"{aspect:<20} {mask.sum():>8} {'N/A':>10} {'N/A':>10}")


PER-ASPECT BREAKDOWN (for comparison with BERT):
Aspect                Samples   Accuracy   Macro-F1
-------------------- -------- ---------- ----------
AMBIENCE                   62     0.8387     0.7567
AUTHENTICITY & LOCAL VIBE       16     0.8750     0.7143
FOOD                      302     0.8510     0.8267
HALAL COMPLIANCE            2     1.0000     1.0000
LOCATION                   21     0.6190     0.5714
LOYALTY (RETURN INTENT)       91     0.8022     0.7831
NON-HALAL ELEMENTS         10     0.7000     0.6703
SERVICE                   128     0.8984     0.8981
VALUE                      88     0.7727     0.7726
nan                         1     1.0000     1.0000


In [18]:
print(f"\nFULL CLASSIFICATION REPORT:")
print(classification_report(y_gold, y_pred_gold, target_names=["Negative", "Positive"], digits=4))


FULL CLASSIFICATION REPORT:
              precision    recall  f1-score   support

    Negative     0.8305    0.7127    0.7671       275
    Positive     0.8371    0.9103    0.8722       446

    accuracy                         0.8350       721
   macro avg     0.8338    0.8115    0.8197       721
weighted avg     0.8346    0.8350    0.8321       721



**Findings**:
- Best model: Logistic Regression will be used for hyperparameter tuning

# Stage 9: Hyperparameter Tuning for Best Model

**Academic Justification:**
- TF-IDF parameters (n-gram range, feature count, min/max df) affect feature space
- Random Forest parameters (n_estimators, max_depth, min_samples) affect model capacity
- Joint tuning ensures optimal feature-model combination
- Using RandomizedSearchCV for efficiency (tests subset of parameter space)
- 3-fold CV on training set prevents overfitting to validation set

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

print("="*70)
print(f"HYPERPARAMETER TUNING FOR BEST MODEL: {best_model_name}")
print("="*70)

# Define parameter distributions based on best model type
if best_model_name == "Random Forest":
    param_distributions = {
        'tfidf__max_features': [3000, 5000, 7000, 10000],
        'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'tfidf__min_df': [2, 3, 5],
        'tfidf__max_df': [0.7, 0.8, 0.9],
        'tfidf__sublinear_tf': [True, False],
        'clf__n_estimators': [100, 200, 300, 500],
        'clf__max_depth': [10, 20, 30, None],
        'clf__min_samples_split': [2, 5, 10],
        'clf__min_samples_leaf': [1, 2, 4],
        'clf__max_features': ['sqrt', 'log2', None]
    }
    model_for_tuning = RandomForestClassifier(class_weight=class_weights_dict, random_state=RANDOM_SEED, n_jobs=-1)
    
elif best_model_name == "XGBoost":
    param_distributions = {
        'tfidf__max_features': [3000, 5000, 7000, 10000],
        'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'tfidf__min_df': [2, 3, 5],
        'tfidf__max_df': [0.7, 0.8, 0.9],
        'tfidf__sublinear_tf': [True, False],
        'clf__n_estimators': [100, 200, 300, 500],
        'clf__max_depth': [3, 5, 7, 9],
        'clf__learning_rate': [0.01, 0.05, 0.1, 0.2],
        'clf__subsample': [0.6, 0.8, 1.0],
        'clf__colsample_bytree': [0.6, 0.8, 1.0]
    }
    model_for_tuning = XGBClassifier(scale_pos_weight=scale_pos_weight, random_state=RANDOM_SEED, n_jobs=-1, eval_metric='logloss')
    
elif best_model_name == "Logistic Regression":
    param_distributions = {
        'tfidf__max_features': [3000, 5000, 7000, 10000],
        'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'tfidf__min_df': [2, 3, 5],
        'tfidf__max_df': [0.7, 0.8, 0.9],
        'tfidf__sublinear_tf': [True, False],
        'clf__C': [0.01, 0.1, 1.0, 10.0, 100.0],
        'clf__penalty': ['l1', 'l2'],
        'clf__solver': ['liblinear', 'saga']
    }
    model_for_tuning = LogisticRegression(max_iter=1000, class_weight=class_weights_dict, random_state=RANDOM_SEED, n_jobs=-1)
    
elif best_model_name == "SVM (Linear)":
    param_distributions = {
        'tfidf__max_features': [3000, 5000, 7000, 10000],
        'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'tfidf__min_df': [2, 3, 5],
        'tfidf__max_df': [0.7, 0.8, 0.9],
        'tfidf__sublinear_tf': [True, False],
        'clf__C': [0.1, 1.0, 10.0, 100.0],
        'clf__kernel': ['linear', 'rbf']
    }
    model_for_tuning = SVC(class_weight=class_weights_dict, random_state=RANDOM_SEED)

# Create pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', model_for_tuning)
])

# Initialize RandomizedSearchCV
print(f"\nInitializing RandomizedSearchCV...")
print(f"  Parameter combinations to test: 50")
print(f"  Cross-validation folds: 3")
print(f"  Scoring metric: Macro-F1")

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=50,
    cv=3,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1,
    random_state=RANDOM_SEED
)

print(f"\nStarting hyperparameter search...")
start_time = time.time()

# Fit on training data
random_search.fit(X_train, y_train)

tuning_time = time.time() - start_time
print(f"\n✓ Hyperparameter tuning complete in {tuning_time:.2f} seconds ({tuning_time/60:.2f} minutes)")

# Extract best parameters
best_params = random_search.best_params_
best_score = random_search.best_score_

print(f"\n{'='*70}")
print(f"BEST PARAMETERS FOUND (via 3-fold CV on training set)")
print(f"{'='*70}")
print(f"Best CV Macro-F1 Score: {best_score:.4f}\n")

print(f"TF-IDF Parameters:")
for param, value in best_params.items():
    if param.startswith('tfidf__'):
        print(f"  {param.replace('tfidf__', ''):<20}: {value}")

print(f"\n{best_model_name} Parameters:")
for param, value in best_params.items():
    if param.startswith('clf__'):
        print(f"  {param.replace('clf__', ''):<20}: {value}")

# Show top 5 parameter combinations
print(f"\n{'='*70}")
print(f"TOP 5 PARAMETER COMBINATIONS")
print(f"{'='*70}")
results_df = pd.DataFrame(random_search.cv_results_)
results_df = results_df.sort_values('rank_test_score')

for i in range(min(5, len(results_df))):
    row = results_df.iloc[i]
    print(f"\nRank {i+1}: CV Macro-F1 = {row['mean_test_score']:.4f} (±{row['std_test_score']:.4f})")
    
    # Print TF-IDF params
    tfidf_params = {k: v for k, v in row.items() if k.startswith('param_tfidf__')}
    print(f"  TF-IDF: ", end="")
    print(", ".join([f"{k.replace('param_tfidf__', '')}={v}" for k, v in list(tfidf_params.items())[:3]]))
    
    # Print model params
    clf_params = {k: v for k, v in row.items() if k.startswith('param_clf__')}
    print(f"  {best_model_name}: ", end="")
    print(", ".join([f"{k.replace('param_clf__', '')}={v}" for k, v in list(clf_params.items())[:3]]))

# Save best estimator (pipeline) for later use
best_pipeline = random_search.best_estimator_
best_vectorizer_tuned = best_pipeline.named_steps['tfidf']
best_clf_tuned = best_pipeline.named_steps['clf']

print(f"\n✓ Tuned pipeline saved to: best_pipeline, best_vectorizer_tuned, best_clf_tuned")

# Stage 10: Final Evaluation with Tuned Model

In [ ]:
print("="*70)
print(f"FINAL EVALUATION: TUNED {best_model_name}")
print("="*70)

# Use tuned pipeline
print(f"Using tuned pipeline from Stage 8...")
print(f"  ✓ Model: {best_model_name}")
print(f"  ✓ Best CV Macro-F1: {random_search.best_score_:.4f}")
print(f"  ✓ Trained on {len(X_train):,} samples")

# Extract tuned components
tuned_vectorizer = best_vectorizer_tuned
tuned_clf = best_clf_tuned

print(f"\nTuned Model Configuration:")
print(f"  TF-IDF max_features:  {tuned_vectorizer.max_features}")
print(f"  TF-IDF ngram_range:   {tuned_vectorizer.ngram_range}")
print(f"  TF-IDF vocabulary:    {len(tuned_vectorizer.vocabulary_):,} terms")

# Show top 3 model-specific parameters
model_params = {k: v for k, v in best_params.items() if k.startswith('clf__')}
for i, (param, value) in enumerate(list(model_params.items())[:3]):
    print(f"  {param.replace('clf__', '')}:  {value}")

# === TEST SET EVALUATION ===
print(f"\n{'='*70}")
print(f"TEST SET EVALUATION (After Tuning)")
print(f"{'='*70}")

y_pred_test_tuned = best_pipeline.predict(X_test)
test_metrics_tuned = compute_metrics(y_test, y_pred_test_tuned)

print(f"\n★ TEST SET RESULTS - Tuned {best_model_name}:")
print(f"  Accuracy:    {test_metrics_tuned['accuracy']:.4f}")
print(f"  Macro-F1:    {test_metrics_tuned['macro_f1']:.4f}")
print(f"    Negative F1: {test_metrics_tuned['neg_f1']:.4f}")
print(f"    Positive F1: {test_metrics_tuned['pos_f1']:.4f}")

print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_test_tuned, target_names=["Negative", "Positive"], digits=4))

# === GOLD STANDARD EVALUATION ===
print(f"\n{'='*70}")
print(f"GOLD STANDARD EVALUATION (After Tuning)")
print(f"{'='*70}")

# Load and prepare gold data (same as Stage 7)
gold_df = pd.read_csv(GOLD_PATH)
gold_df_prep = gold_df.copy()
gold_df_prep.rename(columns={
    "Manual_Aspect": "aspect",
    "Manual_Sentiment": "Sentiment_Label",
}, inplace=True)
gold_df_prep["Sentiment_Label"] = gold_df_prep["Sentiment_Label"].str.lower()

import ast
def parse_aspect(val):
    if isinstance(val, str):
        try:
            parsed = ast.literal_eval(val)
            return parsed if isinstance(parsed, list) else [parsed]
        except (ValueError, SyntaxError):
            return [val]
    elif isinstance(val, list):
        return val
    else:
        return [str(val)]

gold_df_prep["aspect"] = gold_df_prep["aspect"].apply(parse_aspect)
gold_df_exploded = gold_df_prep.explode("aspect").reset_index(drop=True)
gold_df_exploded["label"] = gold_df_exploded["Sentiment_Label"].map(LABEL2ID)
gold_df_exploded['processed_text'] = gold_df_exploded['Segment'].apply(preprocess_text)

# Predict on gold with tuned model
y_gold = gold_df_exploded['label'].values
y_pred_gold_tuned = best_pipeline.predict(gold_df_exploded['processed_text'].values)

gold_metrics_tuned = compute_metrics(y_gold, y_pred_gold_tuned)

print(f"\n★ GOLD TEST SET RESULTS - Tuned {best_model_name}:")
print(f"★ Total samples: {len(gold_df_exploded):,} aspect-segment pairs")
print(f"\nOVERALL PERFORMANCE:")
print(f"  Accuracy:  {gold_metrics_tuned['accuracy']:.4f}")
print(f"  Macro-F1:  {gold_metrics_tuned['macro_f1']:.4f}")
print(f"    Negative F1: {gold_metrics_tuned['neg_f1']:.4f}")
print(f"    Positive F1: {gold_metrics_tuned['pos_f1']:.4f}")

# Per-aspect breakdown
print(f"\nPER-ASPECT BREAKDOWN:")
print(f"{'Aspect':<20} {'Samples':>8} {'Accuracy':>10} {'Macro-F1':>10}")
print(f"{'-'*20} {'-'*8} {'-'*10} {'-'*10}")

aspects_unique = sorted(gold_df_exploded["aspect"].unique())
for aspect in aspects_unique:
    mask = gold_df_exploded["aspect"] == aspect
    y_aspect = y_gold[mask]
    y_pred_aspect = y_pred_gold_tuned[mask]
    
    try:
        metrics = compute_metrics(y_aspect, y_pred_aspect)
        n_samples = mask.sum()
        print(f"{aspect:<20} {n_samples:>8} {metrics['accuracy']:>10.4f} {metrics['macro_f1']:>10.4f}")
    except:
        print(f"{aspect:<20} {mask.sum():>8} {'N/A':>10} {'N/A':>10}")

print(f"\nFULL CLASSIFICATION REPORT:")
print(classification_report(y_gold, y_pred_gold_tuned, target_names=["Negative", "Positive"], digits=4))

print(f"\n{'='*70}")
print(f"✓ FINAL MODEL EVALUATION COMPLETE")
print(f"{'='*70}")